In [3]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /usr/local/lib/python3.12/site-packages/starrynet-1.0.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 33.2 MB/s eta 0:00:00m eta 0:00:01:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [38]:
import os 
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [39]:
base_path = "output"
if os.path.exists(f"{base_path}/connection_peroids.csv"):
    print("Loading existing processed data...")
df_conn_periods = pd.read_csv(f"{base_path}/connection_periods.csv")

In [40]:
# df_conn_periods.iloc[0]['TLE_Line2']

In [41]:
df_conn_periods.columns

Index(['Satellite', 'Start_Time', 'End_Time', 'Duration_Seconds',
       'Max_Distance', 'Min_Distance', 'First_Distance', 'Last_Distance',
       'Mean_Distance', 'Std_Distance', 'TLE_Line1', 'TLE_Line2',
       'TLE_Timestamp'],
      dtype='object')

In [58]:
# how many unique satellite names are there?
satellite_names = df_conn_periods['Satellite'].unique()
satellite_names = [name for name in satellite_names if name != '']
print(f"Unique satellite names: {satellite_names}")
print(f"Number of unique satellite names: {len(satellite_names)}")
satellite_counts = df_conn_periods['Satellite'].value_counts()
satellite_counts = satellite_counts[satellite_counts.index != '']

# Print the most frequently seen satellite
most_seen_satellite = satellite_counts.idxmax()
most_seen_count = satellite_counts.max()

print(f"Most frequently seen satellite: {most_seen_satellite} ({most_seen_count} times)")
print("\nTop 10 satellites by frequency:")
print(satellite_counts.head(10))

Unique satellite names: ['STARLINK-1972', 'STARLINK-5185', 'STARLINK-5507', 'STARLINK-3629', 'STARLINK-30272', 'STARLINK-32697', 'STARLINK-4208', 'STARLINK-32235', 'STARLINK-1848', 'STARLINK-3651', 'STARLINK-32369', 'STARLINK-6046', 'STARLINK-5200', 'STARLINK-5973', 'STARLINK-31458', 'STARLINK-32304', 'STARLINK-1942', 'STARLINK-5171', 'STARLINK-5174', 'STARLINK-31356', 'STARLINK-32289', 'STARLINK-1608', 'STARLINK-31556', 'STARLINK-31140', 'STARLINK-2568', 'STARLINK-5222', 'STARLINK-3835', 'STARLINK-5482', 'STARLINK-4046', 'STARLINK-30525', 'STARLINK-1200', 'STARLINK-32790', 'STARLINK-5412', 'STARLINK-3451', 'STARLINK-5789', 'STARLINK-30540', 'STARLINK-30915', 'STARLINK-32272', 'STARLINK-3540', 'STARLINK-3884', 'STARLINK-30979', 'STARLINK-1275', 'STARLINK-1922', 'STARLINK-5885', 'STARLINK-5151', 'STARLINK-5032', 'STARLINK-4092', 'STARLINK-30534', 'STARLINK-30930', 'STARLINK-5958', 'STARLINK-3645', 'STARLINK-5966', 'STARLINK-32032', 'STARLINK-5138', 'STARLINK-30497', 'STARLINK-5948', 'ST

In [44]:
new_df = df_conn_periods[df_conn_periods['Satellite'] == 'STARLINK-33569']
print(f"Number of rows for STARLINK-5185: {len(new_df)}")
print(new_df['Mean_Altitude_km'])

print(new_df['Start_Time'].nunique())
print(new_df[['Start_Time', 'End_Time']])

plt.figure(figsize=(12, 6))
plt.plot(new_df['Start_Time'], new_df['Mean_popPingLatencyMs'], marker='o', linestyle='-', markersize=5)
plt.xlabel('Start Time')
plt.xticks(rotation=45)
plt.ylabel('Ping Latency (ms)')
plt.title('Ping Latency Over Time for STARLINK-4732')
plt.grid(True)
plt.tight_layout()
plt.show()


Number of rows for STARLINK-5185: 29


KeyError: 'Mean_Altitude_km'

In [59]:
def extract_mean_motion(tle_line2):
    try:
        # Extract characters at positions 53-63 (0-indexed: 52–63, exclusive)
        mean_motion_str = tle_line2[52:63].strip()
        return float(mean_motion_str)
    except Exception:
        return None

# Compute mean motion and orbital periods
df_conn_periods['Mean_Motion'] = df_conn_periods['TLE_Line2'].apply(extract_mean_motion)
df_conn_periods['Orbital_Period_Minutes'] = df_conn_periods['Mean_Motion'].apply(
    lambda n: 1440 / n if pd.notnull(n) and n != 0 else None
)
df_conn_periods['Orbital_Period_Days'] = df_conn_periods['Mean_Motion'].apply(
    lambda n: 1 / n if pd.notnull(n) and n != 0 else None
)

sat = 'STARLINK-32827'
# Preview
# print(df_conn_periods[['Satellite', 'Mean_Motion', 'Orbital_Period_Minutes', 'Mean_Altitude_km']]['Satellite'] == 'STARLINK-4732')
print(df_conn_periods[df_conn_periods['Satellite'] == sat][['Satellite', 'Mean_Motion', 'Orbital_Period_Minutes']])
df = (df_conn_periods[df_conn_periods['Satellite'] == sat]).copy()

# print(df_conn_periods[['Satellite', 'Mean_Motion', 'Orbital_Period_Minutes', 'Mean_Altitude_km']].head())


            Satellite  Mean_Motion  Orbital_Period_Minutes
3216   STARLINK-32827    15.301865               94.106176
3217   STARLINK-32827    15.301865               94.106176
3218   STARLINK-32827    15.301865               94.106176
3219   STARLINK-32827    15.301865               94.106176
3220   STARLINK-32827    15.301865               94.106176
3221   STARLINK-32827    15.301865               94.106176
7303   STARLINK-32827    15.301953               94.105633
7304   STARLINK-32827    15.301953               94.105633
7305   STARLINK-32827    15.301953               94.105633
8170   STARLINK-32827    15.301953               94.105633
8171   STARLINK-32827    15.301953               94.105633
8172   STARLINK-32827    15.301953               94.105633
8173   STARLINK-32827    15.301953               94.105633
12348  STARLINK-32827    15.301895               94.105990
14456  STARLINK-32827    15.301881               94.106079
21234  STARLINK-32827    15.301922               94.1058

In [60]:
from datetime import datetime

# Ensure time columns are datetime
df['Start_Time'] = pd.to_datetime(df['Start_Time'])
df['End_Time'] = pd.to_datetime(df['End_Time'])

# Sort chronologically by Start_Time
df = df.sort_values('Start_Time').reset_index(drop=True)

# Calculate time since previous connection (in minutes)
df['Minutes_Since_Last_Contact'] = df['Start_Time'].diff().dt.total_seconds() / 60

# Calculate number of orbits completed between connections
df['Orbits_Since_Last_Contact'] = df['Minutes_Since_Last_Contact'] / df['Orbital_Period_Minutes']

# Optional: round orbits to nearest integer
df['Orbits_Since_Last_Contact'] = df['Orbits_Since_Last_Contact'].round(2)

print(df[['Minutes_Since_Last_Contact', 'Orbits_Since_Last_Contact']])


    Minutes_Since_Last_Contact  Orbits_Since_Last_Contact
0                          NaN                        NaN
1                     0.266667                       0.00
2                     0.516667                       0.01
3                     0.233333                       0.00
4                     0.250000                       0.00
5                     0.266667                       0.00
6                  1409.000000                      14.97
7                     0.233333                       0.00
8                     0.250000                       0.00
9                   295.000000                       3.13
10                    0.516667                       0.01
11                    0.233333                       0.00
12                    0.250000                       0.00
13                 1408.983333                      14.97
14                 2822.233333                      29.99
15                 2820.766667                      29.97
16            

In [59]:
import datetime
from skyfield.api import load
def get_visible_satellites(timestamp_str, all_satellites, centralLat, centralLon, offsetLat, offsetLon):
    timestamp_dt = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S%z")
    timescale = load.timescale(builtin=True)
    time_ts = timescale.utc(
        timestamp_dt.year,
        timestamp_dt.month,
        timestamp_dt.day,
        timestamp_dt.hour,
        timestamp_dt.minute,
        timestamp_dt.second,
    )

    visible_sats = []

    for sat in all_satellites:
        geocentric = sat.at(time_ts)
        subsat = geocentric.subpoint()
        lat = subsat.latitude.degrees
        lon = subsat.longitude.degrees

        if (
            centralLat - offsetLat * 1.5 < lat < centralLat + offsetLat * 1.5
            and centralLon - offsetLon < lon < centralLon + offsetLon
        ):
            visible_sats.append(sat.name)

    return visible_sats


In [ ]:
from tqdm import tqdm

# Only if tqdm isn't already installed
# !pip install tqdm

visible_satellite_lists = []

# Use tqdm for progress bar during iteration
for _, row in tqdm(df_conn_periods.iterrows(), total=len(df_conn_periods)):
    timestamp_str = row['Start_Time']
    visible_sats = get_visible_satellites(
        timestamp_str,
        all_satellites,
        centralLat=44.97,   # Replace with your actual value
        centralLon=-93.26,  # Replace with your actual value
        offsetLat=5,        # Adjust as needed
        offsetLon=10        # Adjust as needed
    )
    visible_satellite_lists.append(visible_sats)

df_conn_periods['Visible_Satellites'] = visible_satellite_lists
